In [1]:
import pandas as pd
import numpy as np

In [31]:
confirmed = pd.read_csv('../Zaida/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('../Zaida/time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('../Zaida/time_series_covid19_recovered_global.csv')

In [33]:
confirmed = confirmed.melt(id_vars=['Country/Region','Province/State','Lat','Long'],var_name="Date",value_name="Confirmed").sort_values(by=['Country/Region','Date'])
deaths = deaths.melt(id_vars=['Country/Region','Province/State','Lat','Long'],var_name="Date",value_name="Deaths").sort_values(by=['Country/Region','Date'])
recovered = recovered.melt(id_vars=['Country/Region','Province/State','Lat','Long'],var_name="Date",value_name="Recovered").sort_values(by=['Country/Region','Date'])

##### Define JH data and filter out columns and rows that are not important

In [34]:
df = pd.merge(confirmed,deaths,on=['Country/Region','Province/State','Lat','Long','Date'])
df = pd.merge(df,recovered,on=['Country/Region','Province/State','Lat','Long','Date'])
df

26536
19367


##### Add countryinfo data

In [19]:
extra_data_df = pd.read_csv("../country_info.csv")
extra_data_df["Country/Region"] = [country_name.replace("'","") for country_name in extra_data_df["country"]]
extra_data_df['Province/State'] = np.nan
extra_data_df = extra_data_df.drop(['country','Timestamp','ID'],axis=1)
extra_data_df['Province/State'] = extra_data_df['Province/State'].astype('object')
combined = pd.merge(df,extra_data_df,on=['Country/Region','Province/State'])


In [20]:
combined

,Country/Region,Date,Confirmed,Deaths,Recovered,start_date,lockdown_severity,leasure_shopping_prohibited,maximum_together,mentality_grade,...,apr_temp,may_temp,jun_temp,july_temp,aug_temp,sept_temp,oct_temp,nov_temp,dec_temp,annual_temp
0,Belgium,1/22/20,0,0,0,3/18/2020,5.0,1,2.0,7.0,...,8.145482,12.398000,15.434529,17.238529,17.183294,14.427412,10.678388,5.761971,3.113924,9.514241
1,Belgium,1/23/20,0,0,0,3/18/2020,5.0,1,2.0,7.0,...,8.145482,12.398000,15.434529,17.238529,17.183294,14.427412,10.678388,5.761971,3.113924,9.514241
2,Belgium,1/24/20,0,0,0,3/18/2020,5.0,1,2.0,7.0,...,8.145482,12.398000,15.434529,17.238529,17.183294,14.427412,10.678388,5.761971,3.113924,9.514241
3,Belgium,1/25/20,0,0,0,3/18/2020,5.0,1,2.0,7.0,...,8.145482,12.398000,15.434529,17.238529,17.183294,14.427412,10.678388,5.761971,3.113924,9.514241
4,Belgium,1/26/20,0,0,0,3/18/2020,5.0,1,2.0,7.0,...,8.145482,12.398000,15.434529,17.238529,17.183294,14.427412,10.678388,5.761971,3.113924,9.514241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1600,United Kingdom,5/3/20,186599,28446,0,3/23/2020,7.0,1,2.0,7.5,...,6.742380,9.815439,12.765885,14.435399,14.286812,12.213888,9.472010,5.524062,3.849172,8.310239
1601,United Kingdom,5/4/20,190584,28734,0,3/23/2020,7.0,1,2.0,7.5,...,6.742380,9.815439,12.765885,14.435399,14.286812,12.213888,9.472010,5.524062,3.849172,8.310239
1602,United Kingdom,5/5/20,194990,29427,0,3/23/2020,7.0,1,2.0,7.5,...,6.742380,9.815439,12.765885,14.435399,14.286812,12.213888,9.472010,5.524062,3.849172,8.310239
1603,United Kingdom,5/6/20,201101,30076,0,3/23/2020,7.0,1,2.0,7.5,...,6.742380,9.815439,12.765885,14.435399,14.286812,12.213888,9.472010,5.524062,3.849172,8.310239


In [26]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [27]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [28]:
dataframe = pandas.read_csv('../airline-passengers.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
dataset

FileNotFoundError: [Errno 2] No such file or directory: '../airline-passengers.csv'

In [91]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [94]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

96 48


In [96]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [101]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [102]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [103]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

TypeError: Expected int32, got list containing Tensors of type '_Message' instead.